# Scraping the same data from multiple pages

In this exercise, we'll look in multiple Exxon annual reports to find how their proven crude oil reserves have changed over time.

To use this script, you need to download and save the list of report summary URLs acquired by doing a search of the SEC database for the Exxon company code.  The file is on GitHub in the same directory as this script and is called `docs.txt`.

To see a more complicated version of this script that carries out the search of the SEC database for annual reports of a bunch of companies, then captures the names of their boards of directors, see https://github.com/HeardLibrary/digital-scholarship/blob/master/code/scrape/python/scrape_sec.py 

## Setup

As before, import the two necessary modules:

In [ ]:
import requests   # best library to manage HTTP transactions
from bs4 import BeautifulSoup # web-scraping library

Define the function to read in the list of URLs for pages to scrape and hard code some variables.

In [ ]:
def getUrlList(path):
    with open(path, 'rt') as fileObject:
        lineList = fileObject.read().split('\n')
    # remove any extra item caused by trailing newline
    if lineList[len(lineList)-1] == '':
        lineList = lineList[0:len(lineList)-1]
    return lineList

baseUrl = 'https://www.sec.gov'

acceptMediaType = 'text/html'
userAgentHeader = 'BaskaufScraper/0.1 (steve.baskauf@vanderbilt.edu)'
requestHeaderDictionary = {
    'Accept' : acceptMediaType,
    'User-Agent': userAgentHeader
}

## First round of scraping to find the URLs of the annual reports

This is a hack of the script from last week that extracts only the URL of only the 10-K report.

In [ ]:
urlList = getUrlList('docs.txt')

lookupUrls = []
for url in urlList:
    response = requests.get(url, headers = requestHeaderDictionary)
    soupObject = BeautifulSoup(response.text,features="html5lib")

    rowObjects = soupObject.find_all('tr')
    for row in rowObjects:
        cellObjects = row.find_all('td')
        for cell in cellObjects:
            if cell.text == '10-K':
                lookupUrl = baseUrl + cellObjects[2].a.get('href')
                print(lookupUrl)
                lookupUrls.append(lookupUrl)

## Test scrape of the first annual report

The following three cells work out the scrape process using only the first annual report.  By separating the steps of the scrape, we avoid repeated hits on the server and also the delays caused by the slowness of the steps.  

Load the report HTML as a Beautiful Soup object:

In [ ]:
response = requests.get(lookupUrls[0], headers = requestHeaderDictionary)
soupObject = BeautifulSoup(response.text,features="html5lib")

The report year is at the very top of the report, so we just need to go through enough elements to find it.  The `break` command stops the loop after the first bold text that is found.

In [ ]:
paragraphs = soupObject.find_all('p')
for p in paragraphs:
    bold = p.find_all('b')
    if len(bold) != 0:
        year = bold[0].text
        print(year)
        break

The Proved Reserves table is somewhere in the middle of the report.  Rather than stepping through the tables, we do a quick and dirty method of just pulling in every row in every table, then look through the cells in each row until we find the text for the row description "Total Proved".  (We only use "Total Proved" because there is a line break after "Proved".) We can see from the table that the number we want for crude oil is in the second column (i.e. cell with index 1).  

In [ ]:
rowObjects = soupObject.find_all('tr')
for row in rowObjects:
    cellObjects = row.find_all('td')
    for cellObject in cellObjects:
        if "Total Proved" in cellObject.get_text():
            provedBarrels = cellObjects[1].text
            print(provedBarrels)

## Actual scrape of all of the annual reports

Now that we know the scrape is getting us what we want, put the three cells above into a loop that iterates through all of the annual reports.  

The process of retrieving the documents, and parsing and searching the HTML is slow for each document, so print the URL of the document as each one starts to get an indication of progress.

In [ ]:
table = []
for reportUrl in lookupUrls:
    print(reportUrl)
    response = requests.get(reportUrl, headers = requestHeaderDictionary)
    soupObject = BeautifulSoup(response.text,features="html5lib")
    paragraphs = soupObject.find_all('p')
    for p in paragraphs:
        bold = p.find_all('b')
        if len(bold) != 0:
            year = bold[0].text.strip()
            print(year)
            break
    rowObjects = soupObject.find_all('tr')
    for row in rowObjects:
        cellObjects = row.find_all('td')
        for cellObject in cellObjects:
            if "Total Proved" in cellObject.get_text():
                provedBarrels = cellObjects[1].text.strip()
                print(provedBarrels)
    table.append([year, provedBarrels])

For our purposes here, we've just printed the list of lists.  If you wanted to output it to a file, you could use the `writeCsv()` function from last week to save the data in a CSV spreadsheet.

In [ ]:
print(table)